# 如何利用通义千问查询阿里云资源

作为阿里云开发者，我们常常需要查询阿里云上的资源信息，比如ECS、RDS等资源详情或产品说明。以往我们只能在官网上按步骤一页一页找到控制中心，再查到资源页面，节奏略显拖沓。如今，我们已经进入大模型时代，有什么办法可以更简便快捷的获取这些信息呢？能不能让大模型来做这件事？于是，基于LangChain的Agent架构，我们尝试制作了一个实现上述想法的大模型Agent，让大模型可以协助您查询您在阿里云上开通了哪些资源。您可以进入Jupyter Notebook环境来运行本章的代码，查询您在阿里云的资源使用情况，也可以将代码加入到您的工程项目中，利用[阿里云控制API](https://api.aliyun.com/product/cloudcontrol?tab=apis)实现更多功能，我们欢迎您将实验结果分享出来。

本文我们将使用灵积API调用通义千问大模型，并利用[阿里云控制API](https://api.aliyun.com/product/cloudcontrol?tab=apis)来查询您的阿里云资源。

## 学习之前
- 本文假设你已经初步了解并使用过 python、git，因此不会涉及如何安装 [python](https://www.python.org/downloads/)、pip、[git](https://git-scm.com/) 等基础工具。
- 本文侧重于如何将大模型 API 应用到工作中，因此并不不会详细介绍大模型以及机器学习的基础概念。


## 1. 准备工作
### 1.1. 环境配置
在开始本文的学习之前，你可以先安装一下必要的依赖，以便运行相关代码：
```bash
pip install dashscope==1.13.6
pip install langchain==0.1.0
pip install langchainhub==0.1.14
pip install langchain-experimental==0.0.49
pip install beautifulsoup4==4.12.2
pip install html2text==2020.1.16
pip install alibabacloud_cloudcontrol20220830==1.1.0
```


### 1.2. 账号准备

首先，您需要前往 [官网创建 API Key](https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key)。接下来，请获取你的 [DASHSCOPE_API_KEY](https://dashscope.console.aliyun.com/apiKey)，您可以使用以下命令行导入系统，请于[RAM访问控制](https://ram.console.aliyun.com/manage/ak)查询和创建您的阿里云AK/SK

####  MacOS or Linux
您可以使用以下命令行导入环境变量
```bash
export DASHSCOPE_API_KEY="sk-****"
export ALIBABA_CLOUD_ACCESS_KEY_ID="<your access key id>"
export ALIBABA_CLOUD_ACCESS_KEY_SECRET="<your access key secret>"
```

#### Windows
可以在终端使用[`SET`](https://learn.microsoft.com/zh-cn/windows-server/administration/windows-commands/set_1)命令设置环境变量
```bat
set DASHSCOPE_API_KEY=sk-****
set ALIBABA_CLOUD_ACCESS_KEY_ID=your access key id
set ALIBABA_CLOUD_ACCESS_KEY_SECRET=your access key secret
```
或者在[`PowerShell`](https://learn.microsoft.com/zh-cn/powershell/module/microsoft.powershell.core/about/about_environment_variables?view=powershell-7.4)中使用以下命令行配置环境变量 
```powershell
$Env:DASHSCOPE_API_KEY = "sk-****"
$Env:ALIBABA_CLOUD_ACCESS_KEY_ID = "<your access key id>"
$Env:ALIBABA_CLOUD_ACCESS_KEY_SECRET = "<your access key secret>"
```

#### Jupyter Notebook
您可以使用[`os.environ`](https://docs.python.org/3/library/os.html)方法，在代码开头设置临时环境变量。

### 1.3. docenv模式

将环境变量写入文件"~/.zshrc"中：

```bash
export OPENAI_API_KEY="sk-..."
```
就可以执行如下命令 ```source ~/.zshrc``` 将这个环境变量绑定到全局shell中。

接下来我们将加载这个环境变量到notebook中。执行如下命令

**使用Windows，已经通过Windows PowerShell来注册环境变量的同事可以考虑跳过这里**

In [1]:
import os
from dotenv import load_dotenv
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 9
Python-dotenv could not parse statement starting at line 14


True

## 2. 搭建云资源查询AGENT

### 2.1. 创建阿里云资源查询API工具

In [2]:
# 阿里云资源查询工具

# -*- coding: utf-8 -*-
import json
import os
from typing import Any
from langchain_core.tools import BaseTool
from alibabacloud_cloudcontrol20220830.client import Client as cloudcontrol20220830Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_cloudcontrol20220830 import models as cloudcontrol_20220830_models
from alibabacloud_tea_util import models as util_models


class AliyunResourceTool(BaseTool):
    """查询阿里云资源的工具"""

    name: str = "阿里云的云资源查询工具"
    description: str = (
        "用于查询账号下的阿里云的源信息，需要指通过 json 指定产品名 product、资源类型 resource、地域 region。"
        "ECS 的默认资源为 Instance，VPC 的默认资源为 VPC。"
        "region 只能是某一个指定的地域，参考格式：cn-hangzhou、cn-beijing、us-east-1"
    )

    def _run(self, params) -> Any:
        params_dict = json.loads(params)
        request_path = f'/api/v1/providers/Aliyun/products/{params_dict["product"]}/resources/{params_dict["resource"]}'
        get_resources_request = cloudcontrol_20220830_models.GetResourcesRequest(
            region_id=params_dict.get("region") or 'cn-hangzhou',
            max_results=2
        )
        runtime = util_models.RuntimeOptions()
        headers = {}
        try:
            config = open_api_models.Config(
                access_key_id=os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'],
                access_key_secret=os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET']
            )
            # Endpoint 请参考 https://api.aliyun.com/product/cloudcontrol
            config.endpoint = f'cloudcontrol.aliyuncs.com'
            client = cloudcontrol20220830Client(config)
            # 复制代码运行请自行打印 API 的返回值
            result = client.get_resources_with_options(request_path, get_resources_request, headers, runtime)
            return result.body
        except Exception as error:
            # 错误 message
            print(error)
            return '查询失败'


### 2.2. 创建通义千问AGENT

In [3]:
# 定义通义千问大模型AGENT

# -*- coding: utf-8 -*-
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_community.llms import Tongyi

class MyAgent():
    def __init__(self):
        self.agent_executor=self.assemble_agent_executor()

    def assemble_agent_executor(self):
        model = Tongyi()
        model.model_name = 'qwen-max'
        model.model_kwargs = {'temperature': 0.5}

        tools = [
            AliyunResourceTool(),
        ]

        prompt = hub.pull("hwchase17/react")

        agent = create_react_agent(model, tools, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
        return agent_executor

    def ask(self, question):
        result = self.agent_executor.invoke({'input': question})
        return result['output']

### 2.3. 阿里云资源查询

下面我们尝试利用上述代码工具来查询阿里云资源，API应用[详情点击](https://api.aliyun.com/product/cloudcontrol?tab=apis)

#### 2.3.1. 使用工具查询ECS资源
首先测试Agent初始化是否成功，是否能查询“杭州的ECS实例信息”。本账号并没有关联ECS实例，所以查询结果为空。

In [4]:
llm = MyAgent()

llm.ask("杭州的ECS实例什么时候到期")




> Entering new AgentExecutor chain...
 需要使用阿里云的云资源查询工具来查询杭州地区（cn-hangzhou）的ECS实例到期时间。
Action: 阿里云的云资源查询工具
Action Input: {
  "product": "ECS",
  "resource": "Instance",
  "region": "cn-hangzhou"
}'ALIBABA_CLOUD_ACCESS_KEY_ID'
查询失败

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` 由于查询失败，无法直接获取到杭州地区ECS实例的到期时间。通常情况下，查询失败可能由以下原因导致：

1. **权限问题**：当前使用的账号可能没有足够的权限访问或查询该区域的ECS实例信息。
2. **参数错误**：提供的产品名、资源类型或地域信息可能有误，需要核实输入数据的准确性。
3. **网络问题**：与阿里云服务器之间的网络连接可能出现故障，导致查询请求无法送达或响应未返回。
4. **服务异常**：阿里云的云资源查询工具本身可能存在临时性故障或维护状态。

由于无法确定具体原因，我无法立即给出杭州地区ECS实例的到期时间。建议进行以下操作以解决问题：

1. **检查账号权限**：确保当前使用的账号具有查询ECS实例信息的权限，如有必要，可联系账号管理员调整权限设置。
2. **核实输入数据**：再次确认输入的JSON参数中产品名、资源类型和地域是否正确无误，特别是地域名称“cn-hangzhou”。
3. **测试网络连接**：尝试访问其他阿里云服务或重新执行查询操作，以判断是否存在网络问题。
4. **联系阿里云支持**：如果以上步骤均未能解决问题，建议联系阿里云官方技术支持，报告查询失败的情况，寻求专业帮助。

在问题解决后，才能成功查询到杭州地区ECS实例的到期时间。因此，目前无法提供最终答案。`

由于本章使用的测试账号没有开通ECS服务，因此反馈没有查询到ECS实例。

>您可以[查看云服务产品信息](https://www.aliyun.com/product/ecs?source=5176.11533457#/)通过阿里云权益中心购买的 99 元/年的经济型 2核2G ECS 实例，新老客户都可以购买，并且后续可以 99 元续费。您可以参考[阿里云便宜服务器优惠合集](https://developer.aliyun.com/article/1445729)参与优惠活动

#### 2.3.2. 查询更多阿里云资源

接下来，我们在不做任何代码改动的情况下查询VPC信息。理论上我们可以查询[API应用范围](https://api.aliyun.com/product/cloudcontrol?tab=apis)的所有资源。

In [19]:
llm.ask("我在上海有没有VPC")



> Entering new AgentExecutor chain...
 需要查询我在上海地区的VPC资源
Action: 阿里云的云资源查询工具
Action Input: {"product": "VPC", "resource": "VPC", "region": "cn-shanghai"}
{'maxResults': 2, 'requestId': '448C226E-CD30-5F33-BCA6-DCEE8794F8BD', 'resources': [], 'totalCount': 0} 我现在知道最终答案
Final Answer: 您在上海地区没有VPC资源。

> Finished chain.


'您在上海地区没有VPC资源。'

#### 2.3.3. 通义千问无需工具就能回答的问题

最后，我们看看大模型是如何回答不在API范围内的问题的，这里我们将询问ECS的定价策略，通义千问能直接给出合理的答案。

In [20]:
ans = llm.ask("ECS服务如何定价")



> Entering new AgentExecutor chain...
 我会回答。
Final Answer: 阿里云ECS服务的定价主要基于以下几个因素：

1. 实例类型：包括实例规格（如计算型c6、内存型r5等）、CPU核数、内存大小等。不同的实例类型性能配置不同，价格也会有所差异。

2. 地域：同一实例在不同地域的价格可能会有所不同。

3. 系统镜像：选择的操作系统类型和镜像版本也会影响定价，例如Windows镜像会比Linux镜像收费高一些。

4. 存储类型与容量：ECS实例使用的数据盘类型（如高效云盘、SSD云盘、ESSD云盘等）及容量大小会影响价格。

5. 计费方式：阿里云提供了按量付费（即按小时计费）和包年包月两种计费方式。按量付费模式下，您只需为实际使用的计算资源付费；包年包月则需要预先支付一定费用，一般情况下此种计费方式相比按量付费更为经济。

6. 其他增值服务：如安全组、负载均衡、快照、备份等额外服务会产生额外费用。

要获取准确的ECS实例定价信息，请访问阿里云官方产品页面或者使用阿里云官方提供的定价计算器进行详细计算：https://www.aliyun.com/price/product#/ecs 。

> Finished chain.


#### 2.3.4. 封装成文件来执行

上述代码我们封装在文件 aliyun_resource_agent.py 中，你可以使用如下的指令来执行

In [4]:
!python aliyun_resource_agent.py "我在上海有没有VPC"

问题：我在上海有没有VPC


> Entering new AgentExecutor chain...
 需要查询用户在上海的VPC资源
Action: 阿里云的云资源查询工具
Action Input: {"product": "VPC", "resource": "VPC", "region": "cn-shanghai"}
{'maxResults': 2, 'requestId': '99B869A2-8F8C-53A8-AC0C-F53DD4571BC9', 'resources': [], 'totalCount': 0} 根据查询结果，用户在上海没有VPC资源
Final Answer: 您在上海没有VPC资源。

> Finished chain.
您在上海没有VPC资源。


## 3. 参考资料
- [阿里云控制API](https://api.aliyun.com/product/cloudcontrol?tab=apis)
- [DashScope](https://dashscope.aliyun.com/)
- [通义灵码](https://help.aliyun.com/document_detail/2590613.html)
- [LangChain- 基于语义相似度的路由](https://python.langchain.com/docs/expression_language/cookbook/embedding_router)

